In [4]:
import os

import xarray as xr
import cfgrib

xr.backends.list_engines()

{'scipy': <ScipyBackendEntrypoint>
   Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
   Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html,
 'cfgrib': <CfGribBackend>
   Open GRIB files (.grib, .grib2, .grb and .grb2) in Xarray
   Learn more at https://github.com/ecmwf/cfgrib,
 'rasterio': <RasterioBackend>,
 'store': <StoreBackendEntrypoint>
   Open AbstractDataStore instances in Xarray
   Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}

In [64]:
import cfgrib

hrrr_obs_path = '/Users/limingzhou/zhoul/work/energy/grib2/hrrrdata/hrrrsub_2020_01_05_05F0.grib2'

utah_file = '/Users/limingzhou/zhoul/work/energy/grib2/utah/20200105.hrrr.t15z.wrfsfcf00.grib2'

data_file = utah_file

backend_kwargs={'filter_by_keys': {'typeOfLevel': 'heightAboveGround', 'level': 10}}
ds_10m = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs=backend_kwargs)

backend_kwargs['filter_by_keys']['level'] = 2
ds_2m = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs=backend_kwargs)

backend_kwargs['filter_by_keys'] = {'stepType': 'instant','typeOfLevel': 'surface'}
ds_surface = xr.open_dataset(data_file, engine='cfgrib', backend_kwargs=backend_kwargs)


In [70]:
ds_10m['si10']

<xarray.DataArray 'si10' (y: 1059, x: 1799)> Size: 8MB
[1905141 values with dtype=float32]
Coordinates:
    time               datetime64[ns] 8B ...
    step               timedelta64[ns] 8B ...
    heightAboveGround  float64 8B ...
    latitude           (y, x) float64 15MB ...
    longitude          (y, x) float64 15MB ...
    valid_time         datetime64[ns] 8B ...
Dimensions without coordinates: y, x
Attributes: (12/34)
    GRIB_paramId:                             207
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1905141
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            max
    ...                                       ...
    GRIB_name:                                10 metre wind speed
    GRIB_shortName:                           10si
    GRIB_units:                               m s**-1
    long_name:                                10 metre wind speed
    units:                                    m s**-1
    standard_name:                            unknown

In [85]:
ds_2m['r2']

<xarray.DataArray 'r2' (y: 1059, x: 1799)> Size: 8MB
[1905141 values with dtype=float32]
Coordinates:
    time               datetime64[ns] 8B ...
    step               timedelta64[ns] 8B ...
    heightAboveGround  float64 8B ...
    latitude           (y, x) float64 15MB ...
    longitude          (y, x) float64 15MB ...
    valid_time         datetime64[ns] 8B ...
Dimensions without coordinates: y, x
Attributes: (12/34)
    GRIB_paramId:                             260242
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1905141
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                2 metre relative humidity
    GRIB_shortName:                           2r
    GRIB_units:                               %
    long_name:                                2 metre relative humidity
    units:                                    %
    standard_name:                            relative_humidity

In [83]:
ds_surface['blh']

<xarray.DataArray 'blh' (y: 1059, x: 1799)> Size: 8MB
[1905141 values with dtype=float32]
Coordinates:
    time        datetime64[ns] 8B ...
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
    latitude    (y, x) float64 15MB ...
    longitude   (y, x) float64 15MB ...
    valid_time  datetime64[ns] 8B ...
Dimensions without coordinates: y, x
Attributes: (12/34)
    GRIB_paramId:                             159
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1905141
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                Boundary layer height
    GRIB_shortName:                           blh
    GRIB_units:                               m
    long_name:                                Boundary layer height
    units:                                    m
    standard_name:                            unknown

In [ ]:


def grb_to_grid(grb_obj):
    """Takes a single grb object containing multiple
    levels. Assumes same time, pressure levels. Compiles to a cube"""
    n_levels = len(grb_obj)
    levels = np.array([grb_element['level'] for grb_element in grb_obj])
    indexes = np.argsort(levels)[::-1] # highest pressure first
    cube = np.zeros([n_levels, grb_obj[0].values.shape[0], grb_obj[1].values.shape[1]])
    for i in range(n_levels):
        cube[i,:,:] = grb_obj[indexes[i]].values
    cube_dict = {'data' : cube, 'units' : grb_obj[0]['units'],
                 'levels' : levels[indexes]}
    return cube_dict
        
    
     


### Get all grib files from a folder

In [1]:
g_file = pygrib.open(os.path.join(hrrr_obs_path, 'hrrrsub_2020_01_01_00F0.grib2'))

NameError: name 'pygrib' is not defined

In [ ]:


grb = g_file.select(name='Cloud mixing ratio')
grb_cube=grb_to_grid(grb)

### Get the mina and max datetime

### Get the missing hours

### Extract a rectangle, based on the four corners